In [3]:
import pandas as pd
import numpy as np
import os
import glob
import joblib
import shap
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import precision_score, roc_auc_score
from numba import jit
from tqdm import tqdm
import warnings

# 경고 무시
warnings.filterwarnings('ignore')

# --------------------------------------------------------------------------------
# 0. 환경 설정 및 폴더 생성
# --------------------------------------------------------------------------------
BASE_DIR = "trending"
SHAP_DIR = os.path.join(BASE_DIR, "shap_values")
MODEL_DIR = os.path.join(BASE_DIR, "models")

for d in [BASE_DIR, SHAP_DIR, MODEL_DIR]:
    os.makedirs(d, exist_ok=True)

print(f"📂 Working Directory Created: {BASE_DIR}")

# --------------------------------------------------------------------------------
# 1. 라벨링 함수 (Numba 가속) - Typical Price 기반
# --------------------------------------------------------------------------------
@jit(nopython=True)
def calculate_t_value_window(prices):
    n = len(prices)
    if n <= 2: return 0.0
    x = np.arange(n)
    y = prices
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    numerator = np.sum((x - x_mean) * (y - y_mean))
    denominator = np.sum((x - x_mean)**2)
    if denominator == 0: return 0.0
    slope = numerator / denominator
    intercept = y_mean - slope * x_mean
    y_pred = slope * x + intercept
    residuals = y - y_pred
    ssr = np.sum(residuals**2)
    var_error = ssr / (n - 2)
    if var_error <= 1e-9: return 0.0
    se_slope = np.sqrt(var_error / denominator)
    if se_slope == 0: return 0.0
    return slope / se_slope

@jit(nopython=True)
def get_trend_scanning_labels(prices, look_ahead_window):
    n = len(prices)
    t_values = np.zeros(n)
    for i in range(n - look_ahead_window):
        window_prices = prices[i+1 : i+1+look_ahead_window]
        t_values[i] = calculate_t_value_window(window_prices)
    return t_values

# --------------------------------------------------------------------------------
# 2. 데이터 로드 및 Typical Price 계산
# --------------------------------------------------------------------------------
print("⏳ Loading Data...")
df_raw = pd.read_csv("eth_4hour_raw_price.csv")
df_master = pd.read_csv("eth_4hour_master.csv")

# 날짜 통일
df_raw['date'] = pd.to_datetime(df_raw['date'])
df_master['date'] = pd.to_datetime(df_master['date'])

# [핵심 변경] 종가 대신 Typical Price ((H+L+C)/3) 사용
print("🛠 Calculating Typical Price (Reflecting Volatility)...")
h = df_raw['ETH_High'].values.astype(np.float64)
l = df_raw['ETH_Low'].values.astype(np.float64)
c = df_raw['ETH_Close'].values.astype(np.float64)
typical_prices = (h + l + c) / 3.0

look_ahead = 12 # 48시간
t_vals = get_trend_scanning_labels(typical_prices, look_ahead)

df_raw['Future_t_val'] = t_vals
# t-value 임계값: Typical Price는 변동성이 반영되므로 조금 더 보수적으로 2.0 유지
df_raw['Target_Long'] = (t_vals > 2.0).astype(int)   # 확실한 상승
df_raw['Target_Short'] = (t_vals < -2.0).astype(int) # 확실한 하락

# Master 데이터와 병합
print("🔗 Merging with Technical Indicators...")
cols_to_merge = ['date', 'Target_Long', 'Target_Short']
df = pd.merge(df_master, df_raw[cols_to_merge], on='date', how='inner')

# 미래 데이터가 없는 마지막 행들 제거
df = df[df_raw['Future_t_val'] != 0].reset_index(drop=True)
df = df.sort_values('date').reset_index(drop=True)

print(f"✅ Data Prepared: {df.shape}")

# --------------------------------------------------------------------------------
# 3. 유틸리티: 동적 피처 선택 & SHAP 저장
# --------------------------------------------------------------------------------
def select_best_features(X_train, y_train, top_n=40):
    selector = VarianceThreshold(threshold=0.01)
    selector.fit(X_train)
    valid_cols = X_train.columns[selector.get_support()]
    X_sub = X_train[valid_cols]
    
    model = xgb.XGBClassifier(n_estimators=50, max_depth=3, n_jobs=-1, random_state=42, verbosity=0)
    model.fit(X_sub, y_train)
    
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1][:top_n]
    return X_sub.columns[indices].tolist()

def save_shap_values(model, X_test, fold_date_str, folder_path):
    try:
        explainer = shap.TreeExplainer(model)
        if len(X_test) > 500:
            X_sample = X_test.sample(500, random_state=42)
        else:
            X_sample = X_test
            
        shap_values = explainer.shap_values(X_sample)
        
        save_path = os.path.join(folder_path, f"shap_{fold_date_str}.pkl")
        joblib.dump({
            'explainer': explainer, 
            'shap_values': shap_values, 
            'features': X_sample.columns.tolist(),
            'data_sample': X_sample
        }, save_path)
    except Exception as e:
        print(f"⚠️ SHAP Error: {e}")

# --------------------------------------------------------------------------------
# 4. 메인: Dynamic Walk-Forward Ensemble 
# --------------------------------------------------------------------------------
def run_strategy(df, target_col='Target_Long', train_months=12, test_months=1):
    rows_per_month = 30 * 6
    train_size = train_months * rows_per_month
    test_size = test_months * rows_per_month
    
    # [수정] 결과 데이터프레임용 딕셔너리 (길이가 같은 것들만)
    results = {
        'date': [], 'actual': [], 'ensemble_prob': [],
        'xgb_prob': [], 'lgb_prob': [], 'cat_prob': []
    }
    
    # [수정] 피처 로그는 별도 리스트로 관리
    feature_log = []
    
    drop_cols = ['date', 'Target_Long', 'Target_Short']
    feature_candidates = [c for c in df.columns if c not in drop_cols]
    
    current_idx = 0
    total_rows = len(df)
    
    print(f"\n🚀 Start Strategy: {target_col} | Saving to {BASE_DIR}")
    
    pbar = tqdm(total=total_rows - train_size - test_size)
    
    while current_idx + train_size + test_size <= total_rows:
        train_end = current_idx + train_size
        test_end = train_end + test_size
        
        train_df = df.iloc[current_idx : train_end]
        test_df = df.iloc[train_end : test_end]
        
        test_dates = test_df['date']
        test_date_str = test_dates.iloc[0].strftime("%Y%m%d")
        
        X_train_raw = train_df[feature_candidates]
        y_train = train_df[target_col]
        X_test_raw = test_df[feature_candidates]
        y_test = test_df[target_col]
        
        # 1. 동적 피처 선택
        best_feats = select_best_features(X_train_raw, y_train, top_n=40)
        X_train = X_train_raw[best_feats]
        X_test = X_test_raw[best_feats]
        
        # [수정] 피처 로그 저장
        feature_log.append({
            'date': test_date_str,
            'features': best_feats[:10]
        })
        
        # 2. 모델 학습
        model_xgb = xgb.XGBClassifier(n_estimators=200, learning_rate=0.05, max_depth=5, subsample=0.8, n_jobs=-1, random_state=42, verbosity=0)
        model_xgb.fit(X_train, y_train)
        prob_xgb = model_xgb.predict_proba(X_test)[:, 1]
        
        model_lgb = lgb.LGBMClassifier(n_estimators=200, learning_rate=0.05, max_depth=5, verbose=-1, random_state=42)
        model_lgb.fit(X_train, y_train)
        prob_lgb = model_lgb.predict_proba(X_test)[:, 1]
        
        model_cat = CatBoostClassifier(iterations=200, learning_rate=0.05, depth=5, verbose=0, allow_writing_files=False, random_seed=42)
        model_cat.fit(X_train, y_train)
        prob_cat = model_cat.predict_proba(X_test)[:, 1]
        
        # 3. 앙상블
        prob_ensemble = (prob_xgb + prob_lgb + prob_cat) / 3
        
        # 4. SHAP 저장
        save_shap_values(model_xgb, X_test, test_date_str, SHAP_DIR)
        
        # 5. 결과 수집
        results['date'].extend(test_dates)
        results['actual'].extend(y_test)
        results['ensemble_prob'].extend(prob_ensemble)
        results['xgb_prob'].extend(prob_xgb)
        results['lgb_prob'].extend(prob_lgb)
        results['cat_prob'].extend(prob_cat)
        
        current_idx += test_size
        pbar.update(test_size)
        
    pbar.close()
    
    # [수정] 두 개를 반환
    return pd.DataFrame(results), feature_log

# --------------------------------------------------------------------------------
# 5. 실행 및 저장
# --------------------------------------------------------------------------------

# (A) Long 전략 실행 (변수 2개로 받음)
df_results_long, feature_history = run_strategy(df, target_col='Target_Long', train_months=12, test_months=1)

# 결과 파일 저장
result_path = os.path.join(BASE_DIR, "eth_4hour_long_strategy_results.csv")
df_results_long.to_csv(result_path, index=False)

print("\n" + "="*50)
print(f"✅ Analysis Complete! Results saved to: {result_path}")
print(f"✅ SHAP values saved in: {SHAP_DIR}")
print("="*50)

# 피처 변화 로그 출력
print("\n🔍 Recent Top Features Trend:")
for log in feature_history[-3:]: # 최근 3개 구간만
    print(f"📅 {log['date']}: {log['features'][:5]} ...")

# 최종 성과 요약
threshold = 0.7
preds = (df_results_long['ensemble_prob'] > threshold).astype(int)
precision = precision_score(df_results_long['actual'], preds)
print("="*50)
print(f"🏆 Final Precision (Threshold {threshold}): {precision:.4f}")
print(f"📊 Total Trades: {preds.sum()}")
print("="*50)

📂 Working Directory Created: trending
⏳ Loading Data...
🛠 Calculating Typical Price (Reflecting Volatility)...
🔗 Merging with Technical Indicators...
✅ Data Prepared: (17914, 558)

🚀 Start Strategy: Target_Long | Saving to trending


15660it [04:05, 63.91it/s]                                                                                                                                                                                                                      


✅ Analysis Complete! Results saved to: trending/eth_4hour_long_strategy_results.csv
✅ SHAP values saved in: trending/shap_values

🔍 Recent Top Features Trend:
📅 20250817: ['ETH_KP_Ratio', 'AVAX_PRICE_VS_LOW_360p', 'SOL_PRICE_VS_HIGH_360p', 'fred_RRPONTSYD_trend_20d', 'AVAX_Dist_EMA_72_ma180_ratio'] ...
📅 20250916: ['ETH_KP_Ratio', 'XRP_BB_WIDTH', 'AVAX_Dist_SMA_200_ma180_ratio', 'SOL_MACD_72_156_54_Slow_Pct', 'ETH_Dist_SMA_300_ma180_ratio'] ...
📅 20251016: ['ADA_KP_Ratio', 'ETH_MACDS_72_156_54_Slow_Pct', 'DOGE_Dist_SMA_200_ma180_ratio', 'DOT_Price_div_VWMA_50d', 'SOL_Dist_SMA_200_ma180_ratio'] ...
🏆 Final Precision (Threshold 0.7): 0.4255
📊 Total Trades: 1215


In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.feature_selection import VarianceThreshold
from tqdm import tqdm
from numba import jit

# --------------------------------------------------------------------------------
# 1. 라벨링 로직 재정의 (파일을 새로 불러오므로 다시 계산해야 함)
# --------------------------------------------------------------------------------
@jit(nopython=True)
def calculate_t_value_window(prices):
    n = len(prices)
    if n <= 2: return 0.0
    x = np.arange(n)
    y = prices
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    numerator = np.sum((x - x_mean) * (y - y_mean))
    denominator = np.sum((x - x_mean)**2)
    if denominator == 0: return 0.0
    slope = numerator / denominator
    intercept = y_mean - slope * x_mean
    y_pred = slope * x + intercept
    residuals = y - y_pred
    ssr = np.sum(residuals**2)
    var_error = ssr / (n - 2)
    if var_error <= 1e-9: return 0.0
    se_slope = np.sqrt(var_error / denominator)
    if se_slope == 0: return 0.0
    return slope / se_slope

@jit(nopython=True)
def get_trend_scanning_labels(prices, look_ahead_window):
    n = len(prices)
    t_values = np.zeros(n)
    for i in range(n - look_ahead_window):
        window_prices = prices[i+1 : i+1+look_ahead_window]
        t_values[i] = calculate_t_value_window(window_prices)
    return t_values

# --------------------------------------------------------------------------------
# 2. 데이터 로드 및 Target 재생성 (에러 해결 파트)
# --------------------------------------------------------------------------------
print("🔄 Reloading and Regenerating Targets...")

# Raw Price 로드
df_raw = pd.read_csv("eth_4hour_raw_price.csv")
df_raw['date'] = pd.to_datetime(df_raw['date'])

# Typical Price 계산
h = df_raw['ETH_High'].values.astype(np.float64)
l = df_raw['ETH_Low'].values.astype(np.float64)
c = df_raw['ETH_Close'].values.astype(np.float64)
typical_prices = (h + l + c) / 3.0

# 라벨링 다시 수행 (엄격한 기준 t-value 2.5 적용)
look_ahead = 12
t_vals = get_trend_scanning_labels(typical_prices, look_ahead)

df_raw['Target_Long'] = (t_vals > 2.5).astype(int)
df_raw['Target_Short'] = (t_vals < -2.5).astype(int) # ✅ 이제 Target_Short가 생겼습니다!

print("✅ Targets Regenerated Successfully.")

# Master 데이터 로드
df_master = pd.read_csv("eth_4hour_master.csv")
df_master['date'] = pd.to_datetime(df_master['date'])

# Long 결과 로드 (이미 있는 파일)
df_long_results = pd.read_csv("trending/eth_4hour_long_strategy_results.csv")
df_long_results['date'] = pd.to_datetime(df_long_results['date'])

# --------------------------------------------------------------------------------
# 3. Short 전략 학습 및 예측
# --------------------------------------------------------------------------------
print("🚀 Training Short Strategy separately...")

# Short용 데이터셋 병합
cols_to_merge = ['date', 'Target_Short']
df_short_input = pd.merge(df_master, df_raw[cols_to_merge], on='date', how='inner')
df_short_input = df_short_input.sort_values('date').reset_index(drop=True)

# Short 전략 함수
def run_short_strategy_simple(df, train_months=12, test_months=1):
    rows_per_month = 30 * 6
    train_size = train_months * rows_per_month
    test_size = test_months * rows_per_month
    
    results = {'date': [], 'actual': [], 'ensemble_prob': []}
    
    # 피처 필터링 (간단히)
    feature_candidates = [c for c in df.columns if c not in ['date', 'Target_Short']]
    
    current_idx = 0
    total_rows = len(df)
    
    pbar = tqdm(total=total_rows - train_size - test_size)
    
    while current_idx + train_size + test_size <= total_rows:
        train_end = current_idx + train_size
        test_end = train_end + test_size
        
        train_df = df.iloc[current_idx : train_end]
        test_df = df.iloc[train_end : test_end]
        
        X_train = train_df[feature_candidates]
        y_train = train_df['Target_Short']
        X_test = test_df[feature_candidates]
        y_test = test_df['Target_Short']
        
        # 분산 0 제거
        selector = VarianceThreshold(threshold=0.01)
        selector.fit(X_train)
        valid_cols = X_train.columns[selector.get_support()]
        X_train = X_train[valid_cols]
        X_test = X_test[valid_cols]
        
        # XGBoost (가볍게)
        model = xgb.XGBClassifier(n_estimators=100, max_depth=5, n_jobs=-1, random_state=42, verbosity=0)
        model.fit(X_train, y_train)
        prob = model.predict_proba(X_test)[:, 1]
        
        results['date'].extend(test_df['date'])
        results['actual'].extend(y_test)
        results['ensemble_prob'].extend(prob)
        
        current_idx += test_size
        pbar.update(test_size)
        
    pbar.close()
    return pd.DataFrame(results)

# 실행
df_short_results = run_short_strategy_simple(df_short_input)
df_short_results['date'] = pd.to_datetime(df_short_results['date'])

# --------------------------------------------------------------------------------
# 4. Long vs Short 비교 분석
# --------------------------------------------------------------------------------
# 데이터 합치기
df_combined = pd.merge(df_long_results[['date', 'ensemble_prob']], 
                       df_short_results[['date', 'ensemble_prob']], 
                       on='date', 
                       suffixes=('_Long', '_Short'))

# 가격 데이터 붙이기
df_combined = pd.merge(df_combined, df_raw[['date', 'ETH_Close']], on='date', how='inner')

# 상관관계 확인
print("\n⚔️ Correlation between Long & Short Signals:")
corr = df_combined[['ensemble_prob_Long', 'ensemble_prob_Short']].corr().iloc[0,1]
print(f"Correlation: {corr:.4f}")
if corr < -0.3:
    print(">> ✅ Good! Signals are inversely related (Logical).")
elif corr > 0.3:
    print(">> ⚠️ Warning! Signals are moving together (Something is wrong).")
else:
    print(">> ℹ️ Signals are independent.")

# 시각화 (최근 600봉 = 약 3개월)
subset = df_combined.iloc[-10000:]

fig, ax1 = plt.subplots(figsize=(15, 8))

# 가격
ax1.plot(subset['date'], subset['ETH_Close'], color='black', alpha=0.3, label='Price')
ax1.set_ylabel('ETH Price')

# 신호 (Threshold 0.8 이상인 경우만 표시)
long_hits = subset[subset['ensemble_prob_Long'] > 0.8]
short_hits = subset[subset['ensemble_prob_Short'] > 0.8]

ax1.scatter(long_hits['date'], long_hits['ETH_Close'], color='red', marker='^', s=100, label='Long Signal (Prob>0.8)', zorder=5)
ax1.scatter(short_hits['date'], short_hits['ETH_Close'], color='blue', marker='v', s=100, label='Short Signal (Prob>0.8)', zorder=5)

plt.title("Long vs Short Strategy Signals (Prob > 0.8)")
plt.legend()
plt.grid(True, alpha=0.3)

save_path = "trending/signal_comparison.png"
plt.savefig(save_path)
print(f"\n✅ Chart saved to: {save_path}")

🔄 Reloading and Regenerating Targets...
✅ Targets Regenerated Successfully.
🚀 Training Short Strategy separately...


15660it [03:21, 77.66it/s]                                                                                                                                                                                                                      



⚔️ Correlation between Long & Short Signals:
Correlation: -0.2812
>> ℹ️ Signals are independent.

✅ Chart saved to: trending/signal_comparison.png


In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# --------------------------------------------------------------------------------
# 1. 데이터 로드 및 병합
# --------------------------------------------------------------------------------
try:
    # 파일 로드
    df_long = pd.read_csv("trending/eth_4hour_long_strategy_results.csv")
    df_long['date'] = pd.to_datetime(df_long['date'])
    
    # Short 결과가 있다면 로드 (없으면 에러 처리)
    # (만약 df_short_results가 메모리에 없다면 이 부분은 실행 불가하므로 주석 처리하거나,
    #  Short 없이 Long만 테스트하려면 아래 병합 로직을 수정해야 합니다.
    #  여기서는 df_short_results가 있다고 가정하거나, 파일이 있다고 가정합니다.)
    #  실전 팁: Short 파일이 없으면 그냥 Long 파일만 있어도 됩니다.
    
    # 가격 데이터 로드
    df_price = pd.read_csv("eth_4hour_raw_price.csv")
    df_price['date'] = pd.to_datetime(df_price['date'])
    
    # 병합: Long 결과 + 가격 데이터
    df_backtest = pd.merge(df_long[['date', 'ensemble_prob']], df_price[['date', 'ETH_Close']], on='date')
    
    # 컬럼명 통일 (Long Only 테스트이므로 Long 확률만 있으면 됨)
    df_backtest.rename(columns={'ensemble_prob': 'ensemble_prob_Long'}, inplace=True)
    
    # [중요] Short 데이터가 있다면 병합 (청산 로직에 사용됨)
    # df_short가 없다면 'ensemble_prob_Short'를 0으로 채워서 에러 방지
    if 'df_short_results' in locals():
        df_short = df_short_results.copy()
        df_short['date'] = pd.to_datetime(df_short['date'])
        df_backtest = pd.merge(df_backtest, df_short[['date', 'ensemble_prob']], on='date', suffixes=('', '_Short'))
        df_backtest.rename(columns={'ensemble_prob': 'ensemble_prob_Short'}, inplace=True)
    else:
        df_backtest['ensemble_prob_Short'] = 0.0 # Short 정보 없으면 0으로 처리

    df_backtest = df_backtest.sort_values('date').reset_index(drop=True)

    # [수정 1] Next_Return 계산 (이게 없으면 에러남)
    df_backtest['Next_Return'] = df_backtest['ETH_Close'].pct_change().shift(-1)

    # [수정 2] Buy & Hold 수익률 계산
    buy_hold_return = (df_backtest['ETH_Close'].iloc[-2] / df_backtest['ETH_Close'].iloc[0] - 1) * 100

except Exception as e:
    print(f"⚠️ 데이터 로드 중 오류 발생: {e}")
    # 오류 발생 시 더 이상 진행하지 않음

# --------------------------------------------------------------------------------
# 2. 롱 온리(Long-Only) 분석 함수
# --------------------------------------------------------------------------------
def analyze_long_only(df, entry_threshold=0.6):
    df = df.copy()
    
    # 1. 진입: Long 확률 > entry_threshold (기본 0.6)
    df['Signal_Long'] = (df['ensemble_prob_Long'] > entry_threshold).astype(int)
    
    # 2. 청산: Long 확률 < 0.4 OR Short 확률 > 0.7 (위험 감지)
    # (Short 확률이 0이어도 Long 확률 저하로 청산 가능)
    df['Exit_Long'] = ((df['ensemble_prob_Long'] < 0.4) | (df['ensemble_prob_Short'] > 0.7)).astype(int)
    
    position = 0
    positions = []
    
    for i in range(len(df)):
        # 현재 포지션이 없을 때
        if position == 0:
            if df['Signal_Long'].iloc[i] == 1:
                position = 1 # 매수
        
        # 현재 Long 포지션 보유 중일 때
        elif position == 1:
            if df['Exit_Long'].iloc[i] == 1:
                position = 0 # 청산
        
        positions.append(position)
        
    df['Position'] = positions
    
    # 수익률 계산
    df['Strategy_Return'] = df['Position'] * df['Next_Return']
    df['Trade_Action'] = df['Position'].diff().abs()
    df['Cost'] = df['Trade_Action'] * 0.0005 # 수수료 0.05%
    df['Net_Return'] = df['Strategy_Return'] - df['Cost']
    
    df['Equity_LongOnly'] = (1 + df['Net_Return'].fillna(0)).cumprod()
    
    # 최종 성과
    total_ret = (df['Equity_LongOnly'].iloc[-2] - 1) * 100
    mdd = (df['Equity_LongOnly'] / df['Equity_LongOnly'].cummax() - 1).min() * 100
    
    return total_ret, mdd, df

# --------------------------------------------------------------------------------
# 3. 실행 및 결과 출력
# --------------------------------------------------------------------------------
if 'df_backtest' in locals():
    # 임계값 0.6으로 테스트 (진입 장벽 완화)
    ret, mdd, df_res_long = analyze_long_only(df_backtest, entry_threshold=0.6)

    print("="*40)
    print("🚑 Emergency Diagnostic: Long-Only Strategy")
    print("="*40)
    print(f"Long-Only Return : {ret:.2f}%")
    print(f"Long-Only MDD    : {mdd:.2f}%")
    print(f"B&H Return       : {buy_hold_return:.2f}%")
    print("="*40)

    # 차트 그리기
    plt.figure(figsize=(12, 6))
    plt.plot(df_res_long['date'], df_res_long['Equity_LongOnly'], label='Long Only (Thr 0.6)', color='red', linewidth=2)
    plt.plot(df_backtest['date'], (1+df_backtest['Next_Return'].fillna(0)).cumprod(), label='Buy & Hold (ETH)', color='gray', alpha=0.3)
    plt.yscale('log')
    plt.title("Long Only Strategy (Threshold 0.6) vs Market")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
else:
    print("데이터프레임(df_backtest)이 생성되지 않아 분석을 수행할 수 없습니다.")

🚑 Emergency Diagnostic: Long-Only Strategy
Long-Only Return : 109.77%
Long-Only MDD    : -52.66%
B&H Return       : 1736.08%
